# Week 1: Pseudonymization and anonymization

# Introduction

In this first exercise session, you will try several "anonymization" techniques from the lecture on a dataset, and show the limitations of these techniques. In particular, you will first try several approaches to pseudonymise the dataset, and perform attacks on the pseudonymisation. Then, you will develop an algorithm to $k-$anonymise the dataset. Finally, you will show that even $k-$anonymisation is not perfect, as it allows for other attacks, such as a semantic attack.

**Optional exercises**: Some exercises are marked as optional. Please complete the exercise session before coming back to them.

You should try and do these exercises on our own (and not wait for any instructions from the tutors). We are of course available should you have any questions.
_We are still developing and improving this module. For any question, comment or feedback, please feel free to send an email to [florimond@imperial.ac.uk](mailto:florimond@imperial.ac.uk)._

### Description

This is a real dataset of patient records from the German Health Care service in 1984, taken from the Journal of Applied Econometrics.  Attributes of the dataset are given below.  We have added a fictitious 5-digit identifier representing the last 5 digits of the person's phone number in that region: +49 21 801X XXXX.

Source: Riphahn, Wambach, and Million, “Incentive Effects in the Demand for Health Care: A Bivariate Panel Count Data Estimation.”
via. http://people.stern.nyu.edu/wgreene/Econometrics/PanelDataSets.htm

For reference, the meaning of each attribute is:

ID = last 5 digits of the phoner in that region +49 21 801X XXXX  _(The index of the dataset)_<br> 
FEMALE =  female = 1; male = 0<br>
YEAR = calendar year of the observation<br>
AGE = age in years<br>
HANDDUM = handicapped = 1; otherwise = 0<br>
HHNINC =  household nominal monthly net income in German marks / 10000<br>
HHKIDS = children under age 16 in the household = 1; otherwise = 0<br>
EDUC =  years of schooling<br>
MARRIED =  married = 1; otherwise = 0<br>
BLUEC = blue collar employee = 1; otherwise = 0<br>
WHITEC = white collar employee = 1; otherwise = 0<br>
SELF = self employed = 1; otherwise = 0<br>
DOCVIS =  number of doctor visits in last three months<br>
HOSPVIS =  number of hospital visits in last calendar year<br>

### Loading the dataset with Pandas

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load the dataset in pandas making sure you set the id as the index for easy access (using index_col='id')
private_1984 = pd.read_csv("healthcare_data_1984_private.csv", index_col='id')

Let's check that the dataset is properly loaded by printing the first few lines

In [34]:
private_1984.head()

,FEMALE,YEAR,AGE,HANDDUM,WORKING,HHNINC,HHKIDS,EDUC,MARRIED,WORKING.1,BLUEC,WHITEC,SELF,DOCVIS,HOSPVIS
id,,,,,,,,,,,,,,,
98990,0,1984,54,0.000000,1,0.3050,0,15.0,1,1,0,1,0,1,0
32946,1,1984,44,0.000000,0,0.3050,0,9.0,1,0,0,0,0,0,0
47400,1,1984,58,0.168719,0,0.1434,0,11.0,0,0,0,0,0,0,0
50813,0,1984,64,1.000000,1,0.1500,0,10.5,0,1,1,0,0,7,2
43058,0,1984,30,0.000000,0,0.2400,0,13.0,0,0,0,0,0,6,0


# 1. Pseudonymization: secret formulas

In this exercise, you will be working directly with "secret formulas" as a pseudonymization technique.

**Action:** Before you start, we need you to pick a partner for this exercise session (e.g. the person sitting next to you), with whom you will share your _pseudonymised_ data.

### Exercise 1.A | Creating and cracking a "secret formula"

In this exercise, you will be guided to choose your own secret hashing function and use it to pseudonymize every record in the dataset. You will then share two small datasets with your partner. The first dataset will contain both the true IDs and the pseudonyms. This dataset represents the *background knowledge* that an attacker (your partner) might have obtained in various ways. The second dataset will contain only the pseudonymized IDs. Your partner will try to use the backgound knowledge to re-identify all the records in this second dataset.

Most of the steps are already solved. You are required to write something if and only if you see *"action required"* next to the step number, e.g. **Step 2 *(action required)*.** But don't forget to execute *every* code cell (with `Ctrl-Enter`)!

### Pseudonymizing the dataset

**Step 1.** Choose a *polynomial* hash function of degree 2, i.e. a function of the form $a_0 x^2 + a_{1} x + a_2$. For simplicity (as discussed during the lecture), your coefficients $a_i$ must be *integers* between 0 and 500. Don't reveal your function!

**Step 2 *(action required)*.** Define your function and call it `generate_pseudo_id`. Make sure to change the values of `a[0],a[1],a[2]` to the values of your polynomial.

In [35]:
def generate_pseudo_id(phone_number):
    'Takes a phone number (int) x, and returns a0 * x^2 + a1 * x + a2'
    a = np.array([0,0,0]) # initiate array of coefficients
    a[0] =  8 ### YOU CHOOSE THIS!
    a[1] =  6 ### YOU CHOOSE THIS!
    a[2] =  9 ### YOU CHOOSE THIS!
    x = phone_number
    return a[0] * x**2 + a[1] * x + a[2]

**Step 3.** Create a copy of `private_1984` called `private_1984_double`.

In [46]:
private_1984_double = private_1984.copy()

**Step 4.** Pseudonymize every ID in `private_1984_double` using your function. Keep the `id` column as the first column in the dataset and add the column of pseudonymized IDs as the second one. Call it `pseudo_id`.

In [47]:
# pseudonymise every index of each record
pseudonymized_numbers = [generate_pseudo_id(x) for x in private_1984.index]
private_1984_double.insert(0, 'pseudo_id', pseudonymized_numbers)
private_1984_double.head()

,pseudo_id,FEMALE,YEAR,AGE,HANDDUM,WORKING,HHNINC,HHKIDS,EDUC,MARRIED,WORKING.1,BLUEC,WHITEC,SELF,DOCVIS,HOSPVIS
id,,,,,,,,,,,,,,,,
98990,78392754749,0,1984,54,0.000000,1,0.3050,0,15.0,1,1,0,1,0,1,0
32946,8683709013,1,1984,44,0.000000,0,0.3050,0,9.0,1,0,0,0,0,0,0
47400,17974364409,1,1984,58,0.168719,0,0.1434,0,11.0,0,0,0,0,0,0,0
50813,20655992639,0,1984,64,1.000000,1,0.1500,0,10.5,0,1,1,0,0,7,2
43058,14832189269,0,1984,30,0.000000,0,0.2400,0,13.0,0,0,0,0,0,6,0


### Producing and sharing the background knowledge

**Step 5.** Export 20 random records from your pseudonymized dataset to a csv file (`known_records.csv`) and email it to your partner. This will be the set of records for which your partner knows both ID and pseudonymised ID.

In [48]:
private_1984_double.sample(20).to_csv("known_records.csv")

**Step 6.** Export 20 random records from your pseudonymized dataset to a new dataset `sample_records_double`. Create a copy of `sample_records_double` and call it `sample_records_anon`. Remove the `id` column from it. Then save it to a csv file `sample_records_anon.csv` and email it to your partner. Your partner will have to reidentify these records.

In [53]:
# extract 20 random samples
#Export 20 random records from your pseudonymized dataset to a new dataset sample_records_double. 
#Create a copy of sample_records_double and call it sample_records_anon

#private_1984
#private_1984_double (copy and pseudonymized)

sample_records_double = private_1984_double.sample(20)
sample_records_anon = sample_records_double.copy()

# remove the ID column, and make the `pseudo_id` the index
sample_records_anon.set_index('pseudo_id', inplace=True)
sample_records_anon.to_csv("sample_records_anon.csv")

sample_records_double.head(20) # don't show the output to your partner!

,pseudo_id,FEMALE,YEAR,AGE,HANDDUM,WORKING,HHNINC,HHKIDS,EDUC,MARRIED,WORKING.1,BLUEC,WHITEC,SELF,DOCVIS,HOSPVIS
id,,,,,,,,,,,,,,,,
93006,69201486333,0,1984,37,0.0,1,0.2700,1,10.5,0,1,1,0,0,0,0
73291,42973005203,0,1984,44,0.0,1,0.4400,1,10.5,1,1,0,1,0,4,0
17156,2354729633,1,1984,49,0.0,0,0.5200,1,14.5,1,0,0,0,0,1,0
15337,1881880583,1,1984,47,0.0,0,0.1900,1,9.0,1,0,0,0,0,0,0
16763,2248085939,0,1984,48,1.0,1,0.2000,1,10.5,1,1,0,0,1,3,0
47972,18410790113,0,1984,34,0.0,1,0.3000,0,10.5,0,1,0,0,0,1,0
60888,29659153689,1,1984,42,0.0,0,0.3800,1,18.0,1,0,0,1,0,18,1
96764,74906754161,1,1984,41,0.0,0,0.2900,1,10.5,1,0,0,0,0,0,0
14167,1605716123,1,1984,62,0.0,0,0.2000,0,9.0,1,0,0,0,0,4,0


And this is what the output looks like:

In [54]:
sample_records_anon.head(20)

,FEMALE,YEAR,AGE,HANDDUM,WORKING,HHNINC,HHKIDS,EDUC,MARRIED,WORKING.1,BLUEC,WHITEC,SELF,DOCVIS,HOSPVIS
pseudo_id,,,,,,,,,,,,,,,
69201486333,0,1984,37,0.0,1,0.2700,1,10.5,0,1,1,0,0,0,0
42973005203,0,1984,44,0.0,1,0.4400,1,10.5,1,1,0,1,0,4,0
2354729633,1,1984,49,0.0,0,0.5200,1,14.5,1,0,0,0,0,1,0
1881880583,1,1984,47,0.0,0,0.1900,1,9.0,1,0,0,0,0,0,0
2248085939,0,1984,48,1.0,1,0.2000,1,10.5,1,1,0,0,1,3,0
18410790113,0,1984,34,0.0,1,0.3000,0,10.5,0,1,0,0,0,1,0
29659153689,1,1984,42,0.0,0,0.3800,1,18.0,1,0,0,1,0,18,1
74906754161,1,1984,41,0.0,0,0.2900,1,10.5,1,0,0,0,0,0,0
1605716123,1,1984,62,0.0,0,0.2000,0,9.0,1,0,0,0,0,4,0


### Cracking the pseudonymization

**Step 7.** Once you have received `known_records.csv` and `sample_records_anon.csv` from your partner, download them to the same folder of this Jupyter notebook (you can overwrite yours).

**Step 8 *(action required)*.** Now try to reverse engineer your partner's secret function using **his/her** `known_records.csv`! That is, try to find the function your partner used to pseudonymize the IDs. This means that you have to find the coefficients he/she chose. (Hint: you might find `numpy.polyfit` useful)

In [56]:
known_records = pd.read_csv("known_records.csv", index_col='id') # import your partner's known_records.csv
known_ids = known_records.index.values # this is an array with 20 known IDs
pseudo_ids = known_records['pseudo_id'].values # this is the array with their respective 20 pseudo-IDs

x = known_ids
y = pseudo_ids


cracked_coeff = np.polyfit(x, y, 2) ### YOUR CODE HERE

print(cracked_coeff)

[8.         6.         8.99999381]


**Step 9 *(action required)*.** Now you should've successfully found your partner's function. Great! But we are not done yet. Remember that the real secret is not the function, but the real IDs! So there is one last step to complete:

Use the reverse engineered function to re-identify each record in `sample_records_anon.csv`. 

Hint 1: Suppose you found that your partner's hash function is $3 x^2 + 10 x + 7$, and you want to re-identify (for example) the pseudo-ID 63231897340. Then you need to solve the equation $63231897340 = 3 x^2 + 10 x + 7$, and the solution(s) to the equation will be the original ID! To solve the exercise you need to generalize this example.

Hint 2: You might find `numpy.roots` useful.

In [67]:
sample_records_anon = pd.read_csv("sample_records_anon.csv", index_col='pseudo_id')

def reid(anon_id, coeff): # The re-identifying function. In the specific example, anon_id = 63231897340 and coeff = [3,10,7].
    
    ### YOUR CODE HERE
    realCoeff = [coeff[0],coeff[1],coeff[2]-anon_id]
    roots = np.roots(realCoeff)
    return roots[1]



sample_records_anon.index = [reid(anon_id, cracked_coeff) for anon_id in sample_records_anon.index.values] # re-identify every pseudo-ID
sample_records_anon.index.name = 'id'
sample_records_anon.head()
#np.roots([3,10,7-63231897340])


,FEMALE,YEAR,AGE,HANDDUM,WORKING,HHNINC,HHKIDS,EDUC,MARRIED,WORKING.1,BLUEC,WHITEC,SELF,DOCVIS,HOSPVIS
id,,,,,,,,,,,,,,,
93006.0,0,1984,37,0.0,1,0.27,1,10.5,0,1,1,0,0,0,0
73291.0,0,1984,44,0.0,1,0.44,1,10.5,1,1,0,1,0,4,0
17156.0,1,1984,49,0.0,0,0.52,1,14.5,1,0,0,0,0,1,0
15337.0,1,1984,47,0.0,0,0.19,1,9.0,1,0,0,0,0,0,0
16763.0,0,1984,48,1.0,1,0.20,1,10.5,1,1,0,0,1,3,0


**Step 10 _(action required)_.** Check with your partner that you actually re-identified all the records in `sample_records_anon`! (compare your output of step 9 with **his/her** first output of step 6)

### Exercise 1.B (_optional_) | Higher degree polynomials as secret formulas

You might think that considering only polynomials of degree 2 is a big constraint, and that reverse engineering an arbitrary function would be much more challenging. However, while cracking an arbitrary function might be a bit harder, using higher degree polynomials already creates issues.

**Step 1.** Consider the following polynomial: 
$$f(x) = x^6 - 56 x^5 + 1231 x^4 - 13412 x^3 + 75028 x^2 - 199472 x + 203160.$$
Let's define it:

In [ ]:
def big_poly(x):
    a = np.array([1,-56,1231,-13412,75028,-199472,203160])
    return np.polyval(a,x)

**Step 2 *(action required)*.** Evaluate `big_poly` for ten random inputs of 5-digit positive integers. How big are the numbers you get?

In [ ]:
## YOUR CODE HERE


**Step 3 *(action required)*.** Try to evaluate `big_poly` for inputs 2,5,7,12,14,16. What happens? Are we happy with it?

In [ ]:
## YOUR CODE HERE

**Step 4.** Plot `big_poly` for inputs between 2 and 16 to get a better idea of how the function looks like.

In [ ]:
import matplotlib.pyplot as plt
xvals = np.arange(2, 17, 0.1)
nvals = np.arange(2, 17, 1)
plt.plot(xvals, big_poly(xvals), nvals, big_poly(nvals), 'bo');

**Step 5 *(action required)*.** Note that the degree of our polynomial is still rather low (degree 6). Given the results you got in the previous steps, do you think that (really) high-degree polynomials would work well as secret formulas? Why?

_Your answer here._

# 2. Hash functions and lookup tables

### Exercise 2.A | Breaking dataset anonymized with hash functions
In this exercise, you will attack a dataset that has been pseudonymized *without* salt, but using a "secret" has function. You will use a method from the class to break the anonymity of the data, namely using a lookup table.

The following dataset has been anonymized with an unknown hash function in the `hashlib` library in Python. The first step is to figure which hash function was used to encode this dataset.

#### Import the dataset

In [ ]:
import hashlib
import numpy as np
import pandas as pd

In [ ]:
# Import the dataset
anonymized = pd.read_csv("exercice_2_a_anonymized_dataset.csv") 

# Have a look
anonymized.head()

#### Creating lookup tables
**Step 1 *(action required)*.** Guess which hash function has been used to hash the original IDs. (Hint: look at the [hashlib's document](https://docs.python.org/3/library/hashlib.html) and compare the available hash functions with the form of the pseudonyms in the datasets)

In [ ]:
## YOUR CODE AND ANSWER HERE

**Step 2 *(action required)*.** Now you know which hash function has been used. Suppose you have somehow discovered that the original IDs were 5-digit numbers (from 00000 to 99999). Create a function `create_lookup_table` that takes the length of the original ids as parameter and returns the complete lookup table. This lookup table is a _dictionary_ where the values are the original IDs and the keys are their hash, hence allowing to recover the corresponding ID from the hashed ID.

We will later use the lookup table to bruteforce the hashed ids. 

_Hints: _

1. To fill an integer with leading zeroes, you can use the method `zfill` of strings.
2. To hash a string `s` with a hash function `hf`, you can do: `hashlib.hf(s.encode()).hexdigest()`.

In [ ]:
def create_lookup_table(N):
    
    ### YOUR CODE HERE
    pass


In [ ]:
# create the table here
my_lookup_table = create_lookup_table(5)

#### Using the lookup table
**Step 3 *(action required)*.** Let us now check that your intuition from **Step 1** is correct. For the lookup table you created, calculate the percentage of the hashed ids in the dataset that are also in this table. Create a function `intersection_size` that takes as parameters the dataset and the lookup table and returns this percentage. If you correctly identified this number, this function should thus return 100.

In [ ]:
def intersection_size(dataset,lookup_table):
        
    ### YOUR CODE HERE
    pass


intersection_size(anonymized,create_lookup_table(5))

#### Reversing the hashed id
**Step 4 *(action required)*.** Now that you know which lookup table to use, use it to reverse the hashed ids. Create a function `break_dataset` that takes the dataset and the lookup table as argument and outputs the deanonymized dataset.

In [ ]:
def break_dataset(dataset,lookup_table):

    # YOUR CODE HERE
    pass


break_dataset(anonymized,create_lookup_table(5))

### Exercise 2.B (optional) | Breaking dataset anonymized with md5 and short salt

In the last section, you have successfully re-identify a dataset that was protected with only a hash function. Because this is not immune to bruteforcing as you have seen by yourself, a good practice is to salt it (with a long salt).

The following dataset has been anonymized with the md5 hash function in the `hashlib` library in Python and a short salt (length 1) added anywhere in the initial id. You are going to adapt the previous method to re-identify the dataset despite the short salt.

#### Import the dataset

In [ ]:
# Import the dataset
anonymized = pd.read_csv("exercice_2_b_anonymized_dataset.csv") 

# Have a look
anonymized.head()

#### Creating the lookup table
**Step 1 *(action required)*.** Breaking a salted MD5 hash is more difficult because of the increased number of combinations but the method is the same as earlier. You will have to create a lookup table for each possible position of the salt and each number between 0 and 9. Create a function `create_lookup_table` that takes the length of the ids, the position of the salt and the salt itself as parameters and returns the lookup table corresponding to this salt position and salt value. 

In [ ]:
def create_lookup_table(N,position,salt):

    # YOUR CODE HERE
    pass


#### Finding the correct table
**Step 2 *(action required)*.** Same as before, find the table that contains 100% of the hashed ids. This indirectly gives you the position of the salt and its value. Reuse the `intersection_size` function of exercice 2.a and the previous function to define a function `find_lookup_table` taking the anonymized dataset as parameters and returning the correct lookup table.

In [ ]:
def find_lookup_table(dataset):
    
    ## YOUR CODE HERE
    pass



table = find_lookup_table(anonymized)

#### Using the lookup table
**Step 3 *(action required)*.** Now that you have the correct lookup table, use it to reverse the salted hashed ids as before. Use the `break_dataset` function defined in exercice 2.a and the lookup table of the previous question to deanonymize the dataset.

In [ ]:
## YOUR CODE HERE

# 3. Hashing with long salt (optional)

### Exercise 3 (optional) | Breaking dataset anonymized with md5 and long salt
In real life situation, properly pseudonymized datasets use long salts of unknown length. 

Let's assume that you are in the ideal attack situation: 
- you know the length of the salt (a standard string of 255 lowercase letters here)
- you know the hash function (md5)
- you know a couple of tuples (real id, hashed id)
- you know the position of the salt (appended at the end of the real id) 

Note that in general it might be hard for an attacker to gain this background information. However, a fundamental principle in computer security is that _security through obscurity_ is not a robust approach (see also [Kerckhoffs's principle](https://en.wikipedia.org/wiki/Kerckhoffs%27s_principle)).

The only thing that you don't know is the salt itself. Adapting what has been done in exercice 2 and knowing all this external information to reduce the searching space, find the salt to deanonymize the dataset.

#### Import the dataset

In [ ]:
# Import the dataset
anonymized = pd.read_csv("exercice_3_anonymized_dataset.csv") 

# Have a look
anonymized.head()

#### Finding the salt
**Step 1 *(action required)*.** Create a function `find_salt` that takes the dataset, the id_known and the corresponding_hashed_ids as parameters and returns the salt. Run it to get the salt.

In [ ]:
# External knowledge
id_known = [75721, 80184, 19864, 76699, 92991]
corresponding_hashed_ids = ['4970daee1c3236a77bf045577f0ee62b',
        '4f8bcc5db2565c5a96d4452ed9edbaca',
        'e9a579e5e070bc9a339c0b1819088628',
        '61594b8066fbb7fab77bc53e098e5f7c',
        '484a013245c1929c8337c7cddb745ba0']


def find_salt(dataset,id_known,corresponding_hashed_ids):
    for salt in itertools.product(string.ascii_lowercase,repeat=255):  

        ## YOUR CODE HERE
        pass
        


salt = find_salt(anonymized,id_known,corresponding_hashed_ids)

**Step 2 *(action required)*.** How long is it taking? How many strings does your code have to try for the salt? Do you think you will ever find the salt?

_Your answer here._

## Exercise 4 | k-anonymizing a dataset

In this exercise, you will take a (correctly) pseudonymised dataset, and $k-$anonymise it to protect against re-identification attacks based on quasi-identifiers. You will then perform an attack to show the limit of $k-$anonymity.

In the dataset we consider, the meaning of each attribute is:

`id` = hashed (pseudonymous) id <br>
`gender` = male or female <br>
`education` = the level of education, 0 = highschools, 1 = bachelors, 2 = masters, 3 = PhD<br>
`age` = age in years<br>
`vote` = Conservative, Labour or other<br>

**Take a moment to think about each attribute and figure if it is a quasi-identifier or sensitive information.**

### Loading the data

In [ ]:
import pandas as pd
import numpy as np
import hashlib
import random
import matplotlib.pyplot as plt
import collections

In [ ]:
dataset1  = pd.read_csv("exercise4.csv")
dataset1.head()

### 4.A | Anonymizing the dataset

Firstly, let's inspect the dataset and compute the equivalence classes. Then, we will perform the $k-$anonymization by removing rows belonging to small equivalence classes (this works because our dataset is not very rich, and a lot of records are naturally part of a large equivalence class).

#### 4.A.i | Inspecting the equivalence classes

**Step 1 *(action required)*.** We want to take a look at the equivalence classes and their number of elements in order to k-anonymize the dataset. 

Start by defining a function `size_equivalence_classes` that takes a dataset as parameter and returns the list of the sizes of its equivalence classes.

_Hint: Use the `groupby` function._

In [ ]:
def size_equivalence_classes(dataset):
    '''Returns the list of sizes of the equivalence classes in the dataset.'''
    
    # YOUR CODE HERE
    pass
    

**Step 2.** Now let's plot the distribution of equivalence classes as a histogram.

In [ ]:
# Run this cell to see the results of the plot

# compute the size of equivalence classes
nb = size_equivalence_classes(dataset1)

# plot a histogram
plt.figure(figsize=(15,10))
a = plt.hist(nb, bins=max(nb)-1, align='left', width=.9)
print(a)
plt.xlabel("Size of Equivalence Classes") # with nice axis and title!
plt.ylabel("Number of Equivalence Classes")
plt.title("Number of Equivalence Classes by Size of Equivalence Classes")
plt.xticks(range(max(nb))); # for ticks on the x axis to go from 1 to max

#### 4.A.ii | Removing equivalence classes

Suppose that we want to 4-anonymize this dataset by _only removing rows_ (no generalisation, column suppression, etc). What is the minimum number of rows that you would need to remove in order to assure 4-anonymity ? 

**Step 3 *(action required)*.** Using the previous function to create the dictionary of the number of equivalence classes, define a function `nb_to_remove` that takes k and the dataset as parameters and returns the minimum number of rows to remove in order to assure k-anonymity of the dataset.

In [ ]:
def nb_to_remove(k,dataset):
    
    lengths = size_equivalence_classes(dataset)

    # YOUR CODE HERE
    pass


nb_to_remove(4,dataset1)

Now we want to find _which rows_ need to be removed in order to achieve a given level of anonymity k.

**Step 4 *(action required)*.** Define a function `to_be_removed` that takes k and the dataset as parameters and returns the list of rows to remove. (You should check that the length of this list is the same as the result of the previous question)

_Hint: use `groupby` again, and the `.index` attribute of a `pandas.DataFrame` to obtain recods._

In [ ]:
def to_be_removed(k,dataset):
    
    result = []
    
    ## YOUR CODE HERE
                
    return result

**Step 5 *(action required)*.** It is now time to actually remove these rows. Define a function `remove_rows` taking as parameters k and the dataset and returning the k-anonymized dataset, that uses `to_be_removed` to identify the rows to remove. In order to keep the original one intact, work on a copy of it.

(Hint: the `drop` method of a pandas DataFrame is quick enough for a small dataset like this one)

In [ ]:
def remove_rows(k,dataset):
    
    result = dataset.copy()
    
    # YOUR CODE HERE
    
    return result

**Step 6 _(action required)_** Check that your code works by plotting the size of the equivalence classes of a k_anonymized dataset (you choose the `k`).

In [ ]:
k = ### YOUR CHOICE

k_anonymized = remove_rows(k,dataset1)

In [ ]:
# Run this cell (again) to see the results of the plot

# compute the size of equivalence classes
nb = size_equivalence_classes(k_anonymized)

# plot a histogram
plt.figure(figsize=(15,10))
plt.hist(nb, bins=max(nb)-1-min(nb), align='left', rwidth=.9)
plt.xlabel("Size of Equivalence Classes") # with nice axis and title!
plt.ylabel("Number of Equivalence Classes")
plt.title("Number of Equivalence Classes by Size of Equivalence Classes")
plt.xticks(range(max(nb))); # for ticks on the x axis to go from 1 to max

### 4.B | Homogeneity attacks

We want to check whether a k-anonymized dataset is vulnerable to any _homogeneity attack_ (that is the case when at least one of the equivalence classes has the same sensitive attribute value accross its rows). This will be used to check the dataset you just 4-anonymized in the previous section.

**Step 1 *(action required)*.** Define a function `homogeneity_attackable` that takes the anonymized dataset as parameter and return a boolean indicating whether an homogeneity attack is possible or not on the anonymized dataset. 

In [ ]:
def homogeneity_attackable(dataset):
    
    # YOUR CODE HERE
    pass


**Step 2 *(action required)*.** The value of k chosen earlier directly impacts the level of protection against homogeneity attacks. You can play with this function and the `remove_rows` function when you are done with exercise 2 to check it yourself. What is the smallest `k` for which the dataset is safe? How many records are left after that?

In [ ]:
k = ### YOUR CHOICE HERE

k_anonymized = remove_rows(k,dataset1)

print('records left:', len((k_anonymized)))

homogeneity_attackable(k_anonymized)

## Exercise 5 | Semantic attacks

In this exercise, we are providing you with another dataset that has been properly pseudonymized and 3-anonymized. We want to make sure that a semantic attack is not possible before publishing it. 

#### Load the dataset

In [ ]:
dataset2 = pd.read_csv('exercise5.csv')
dataset2.head()

#### A function to detect semantic similarity 

A _semantic attack_ uses real-world knowledge of the private attribute to learn something about users in the dataset. If all people in an equivalence class have the word `cancer` in their `disease` field, then clearly the attacker can learn something about the people in this class.

**Step 1.** We define a function `too_close` that takes a list of strings (typically diseases names here) `string_list` and a list of stop words (i.e. words that do not give any information, such as 'disease' in this case) `stop_words` as parameters, and returns a boolean indicating whether all the strings in `string_list` all share at least one word, and that word is not in the `stop_word` list. For instance `string_list = ["skin cancer","prostate cancer"]` and `stop_words = ['disease']` should return True, but if `stop_words=['cancer']`, it should return False. For simplicity, the function is already defined, but you should read the code and understand how it works (it uses [sets](https://snakify.org/en/lessons/sets/)).

In [ ]:
def too_close(string_list,stop_words):
    '''Returns True if all strings in string_list share a word, that is not part of stop words'''
    
    # defensive programming: ensure correct input format (can be passed any iterable)
    string_list = list(string_list)
    stop_words  = list(stop_words)
    
    if not string_list:
        return False # no string in the list!

    # create the initial bag of shared words from the first string
    words = set(string_list[0].lower().split())
    
    # remove all stop words from this list (they can't be part of the intersection)
    words.difference_update( set(stop_words) )
        
    for s in string_list[1:]:
        # compute the intersection of words shared by previous strings and the string s
        words.intersection_update(set(s.lower().split()))
    
    # do all strings share at least one non-stop word?
    return len(words) > 0

In [ ]:
# test 1
too_close(['skin cancer', 'prostate cancer'], ['disease'])

In [ ]:
# test 2
too_close(['skin cancer', 'prostate cancer'], ['cancer'])

**Step 2 *(action required)*.** Now define a function `semantically_attackable` that takes as a parameter an anonymized dataset and returns a boolean indicating whether or not that dataset is semantically attackable. Treat the word `disease` as a stop word (as all people in the dataset have a disease, it is not informative).

_Hint: adapt exercise 4.B._

In [ ]:
def semantically_attackable(dataset):
    
    # YOUR CODE HERE
    pass


#### Detecting vulnerability to semantic attacks
**Step 3.** Use this function to check whether or not this dataset is semantically attackable. Is it possible to perform homogeneity attack on this dataset ?

In [ ]:
homogeneity_attackable(dataset2)

In [ ]:
semantically_attackable(dataset2)

**Step 4 *(action required)*.** What do you think about the method we used to check whether the elements of an equivalence class are too close to each other? Does it generalise well to other problems? Is it too restrictive/not enough restrictive?

_Your answer here._

## Exercise 4 - second round (optional)

#### 4.A.iii | Analysing the k-anonymized dataset

When k-anonymizing datasets it is important to keep in mind the use case and the utility of your data. If you anonymize it too harshly, the level of privacy protection is indeed better but the usability of the data might really go down. 

**Step 1.** Lets take a look back at the dataset you 4-anonymized earlier.

In [ ]:
remove_rows(4,dataset1).head()

**Step 2.** Use the function value_counts in pandas and the plot method to check the result of the election before and after anonymization.

In [ ]:
pd.value_counts(dataset1['vote']).plot.bar()
print(pd.value_counts(dataset1['vote']))

In [ ]:
pd.value_counts(remove_rows(4,dataset1)['vote']).plot.bar()
print(pd.value_counts(remove_rows(4,dataset1)['vote']))

**Step 3 *(action required)*.** What happens? Can you imagine some use cases where the k-anonymization we employed might be an issue for analysts who have access only to the k-anonymized dataset?

_Your answer here._